In [1]:
import censusgeocode as cg
import requests
import json
import pandas as pd
from datetime import datetime

In [2]:
brewerydburl = 'https://api.openbrewerydb.org/breweries?per_page=50&page={}'
brewerydata = []
page=1

while True:
    result = requests.get(brewerydburl.format(page)).json()
    if len(result) == 0:
        break
    page += 1
    brewerydata.extend(result)

brewerydata

[{'id': 2,
  'name': 'Avondale Brewing Co',
  'brewery_type': 'micro',
  'street': '201 41st St S',
  'address_2': None,
  'address_3': None,
  'city': 'Birmingham',
  'state': 'Alabama',
  'county_province': None,
  'postal_code': '35222-1932',
  'country': 'United States',
  'longitude': '-86.774322',
  'latitude': '33.524521',
  'phone': '2057775456',
  'website_url': 'http://www.avondalebrewing.com',
  'updated_at': '2018-08-23T23:19:57.825Z',
  'created_at': '2018-07-24T01:32:47.255Z'},
 {'id': 44,
  'name': 'Trim Tab Brewing',
  'brewery_type': 'micro',
  'street': '2721 5th Ave S',
  'address_2': None,
  'address_3': None,
  'city': 'Birmingham',
  'state': 'Alabama',
  'county_province': None,
  'postal_code': '35233-3401',
  'country': 'United States',
  'longitude': '-86.7914000624146',
  'latitude': '33.5128492349817',
  'phone': '2057030536',
  'website_url': 'http://www.trimtabbrewing.com',
  'updated_at': '2018-08-23T23:20:31.423Z',
  'created_at': '2018-07-24T01:32:47.81

In [3]:
brewnames = []
brewtypes = []
brewlats = []
brewlongs = []
brewindex = 0

for brewery in brewerydata:
    brewnames.append(brewerydata[brewindex]["name"])
    brewtypes.append(brewerydata[brewindex]["brewery_type"])
    brewlats.append(brewerydata[brewindex]["latitude"])
    brewlongs.append(brewerydata[brewindex]["longitude"])
    brewindex += 1
    
print(brewnames)

['Avondale Brewing Co', 'Trim Tab Brewing', 'Yellowhammer Brewery', 'Bearpaw River Brewing Co', 'King Street Brewing Co', '1912 Brewing', 'Bad Water Brewing', 'BJs Restaurant & Brewery - Chandler', 'BlackRock Brewers', 'Dragoon Brewing Co', 'Grand Canyon Brewing Company', 'Mudshark Brewing Co', 'Richter Aleworks', 'SanTan Brewing Co', 'State 48 Brewery', 'Wren House Brewing Company', 'Brick Oven Pizza Co / Brick & Forge Brewing', 'Diamond Bear Brewing Co', 'Lost Forty Brewing', "Rapp's Barren Brewing Company", 'Whistleing Springs Brewing Company / Dark Hills Brewery', '7 Sisters Brewing Co', 'Abnormal Beer Company', 'Altamont Beer Works', 'Auburn Alehouse', 'Back Street Brewery & Tasting Room', 'Bear Republic Brewing Co Pub & Restaurant - Lakeside', 'Bird Street Brewing', 'Black Hammer Brewing', 'Brewbakers Brewing Co', 'Brewery in Planning - Los Angeles', 'Brewery in Planning - San Diego', 'Brewery Twenty Five', 'Camino Brewing Co LLC', 'Chula Vista Brewery', 'Band of Brothers Brewing

In [4]:
brewery_df = pd.DataFrame({
    "BreweryName": brewnames,
    "BreweryType": brewtypes,
    "Latitude": brewlats,
    "Longitude": brewlongs
})

brewery_df_filter = brewery_df[brewery_df["Latitude"] != ""]
brewery_df_filter = brewery_df_filter.dropna(axis=0)
brewery_df_filter.reset_index(drop=True,inplace=True)

brewery_df_filter["Latitude"] = brewery_df_filter["Latitude"].apply(float)
brewery_df_filter["Longitude"] = brewery_df_filter["Longitude"].apply(float)

brewery_df_filter

,BreweryName,BreweryType,Latitude,Longitude
0,Avondale Brewing Co,micro,33.524521,-86.774322
1,Trim Tab Brewing,micro,33.512849,-86.791400
2,Yellowhammer Brewery,micro,34.727752,-86.593201
3,Bearpaw River Brewing Co,micro,61.575269,-149.412710
4,King Street Brewing Co,micro,61.138489,-149.879076
...,...,...,...,...
4744,Public Craft Brewing Co,micro,42.583154,-87.819751
4745,Stevens Point Brewery,regional,44.510263,-89.573981
4746,Titletown Brewing Co,brewpub,44.518920,-88.018517
4747,Black Tooth Brewing Co,micro,44.800895,-106.953506


In [5]:
brewcounties = []
brewstates = []
brewcountycodes = []

breweries_without_results = []

for index, row in brewery_df_filter.iterrows():
    x = row['Longitude']
    y = row['Latitude']
    breweries_without_results.append((index, x, y))

brewery_api_results = {}

t1 = datetime.now()
attempt = 1
debug = False
while len(breweries_without_results) > 0 and attempt <= 40:
    breweries_with_errors = []

    for index, x, y in breweries_without_results:
        try:
            api_result = cg.coordinates(x=x, y=y)
            county = api_result['Counties'][0]['BASENAME']
            state = api_result['States'][0]['STUSAB']
            countycode = api_result['Counties'][0]['STATE'] + api_result['Counties'][0]['COUNTY']
            brewery_api_results[index] = (county, state, countycode)
        except Exception as e:
            breweries_with_errors.append((index, x, y))
            
    print((len(breweries_without_results),len(breweries_with_errors)))        
    breweries_without_results = breweries_with_errors
    attempt += 1
            
print("Still " + str(len(breweries_without_results)) + " wo results")

t2 = datetime.now()
print(t2-t1)


brewery_api_results

# LLindex = 0

# for index, row in brewery_df_filter.head().iterrows():
#     print(index)
#     print(row)
#     x = row["Longitude"]
#     y = row["Latitude"]
#     try:
#         brewlocationdata = cg.coordinates(x=x,y=y)
#         brewcounty = brewlocationdata['Counties'][0]['BASENAME']
#         brewstate = brewlocationdata['States'][0]['STUSAB']
#         brewcounties.append(brewcounty)
#         brewstates.append(brewstate)
#     except:
#         brewcounties.append("")
#         brewstates.append("")
#     LLindex += 1
    
# brewcounties

(4749, 1471)
(1471, 449)
(449, 153)
(153, 43)
(43, 15)
(15, 4)
(4, 2)
(2, 0)
Still 0 wo results
1:20:26.364545


{4: ('Anchorage', 'AK', '02020'),
 5: ('Pima', 'AZ', '04019'),
 7: ('Maricopa', 'AZ', '04013'),
 8: ('Pima', 'AZ', '04019'),
 9: ('Pima', 'AZ', '04019'),
 10: ('Coconino', 'AZ', '04005'),
 12: ('Maricopa', 'AZ', '04013'),
 13: ('Maricopa', 'AZ', '04013'),
 14: ('Maricopa', 'AZ', '04013'),
 15: ('Maricopa', 'AZ', '04013'),
 16: ('Greene', 'AR', '05055'),
 17: ('Pulaski', 'AR', '05119'),
 18: ('Pulaski', 'AR', '05119'),
 19: ('Baxter', 'AR', '05005'),
 20: ('Benton', 'AR', '05007'),
 21: ('San Luis Obispo', 'CA', '06079'),
 23: ('Alameda', 'CA', '06001'),
 24: ('Placer', 'CA', '06061'),
 28: ('San Francisco', 'CA', '06075'),
 29: ('Tulare', 'CA', '06107'),
 30: ('Los Angeles', 'CA', '06037'),
 31: ('San Diego', 'CA', '06073'),
 32: ('San Benito', 'CA', '06069'),
 33: ('Santa Clara', 'CA', '06085'),
 34: ('San Diego', 'CA', '06073'),
 36: ('Juneau', 'AK', '02110'),
 38: ('Los Angeles', 'CA', '06037'),
 39: ('San Diego', 'CA', '06073'),
 40: ('Los Angeles', 'CA', '06037'),
 42: ('San Diego

In [6]:
brewstates = []
brewcounties = []
brewcountycodes = []

for i, (county, state, countycode) in sorted(brewery_api_results.items()):
    brewcounties.append(county)
    brewstates.append(state)
    brewcountycodes.append(countycode)
    
brewcounties


['Jefferson',
 'Jefferson',
 'Madison',
 'Matanuska-Susitna',
 'Anchorage',
 'Pima',
 'Maricopa',
 'Maricopa',
 'Pima',
 'Pima',
 'Coconino',
 'Mohave',
 'Maricopa',
 'Maricopa',
 'Maricopa',
 'Maricopa',
 'Greene',
 'Pulaski',
 'Pulaski',
 'Baxter',
 'Benton',
 'San Luis Obispo',
 'San Diego',
 'Alameda',
 'Placer',
 'Orange',
 'Sonoma',
 'Kings',
 'San Francisco',
 'Tulare',
 'Los Angeles',
 'San Diego',
 'San Benito',
 'Santa Clara',
 'San Diego',
 'Tuscaloosa',
 'Juneau',
 'Sonoma',
 'Los Angeles',
 'San Diego',
 'Los Angeles',
 'Alameda',
 'San Diego',
 'San Bernardino',
 'Alameda',
 'San Francisco',
 'Sacramento',
 'Santa Clara',
 'Santa Clara',
 'San Bernardino',
 'Santa Clara',
 'San Joaquin',
 'Los Angeles',
 'Marin',
 'Yolo',
 'San Diego',
 'Los Angeles',
 'Orange',
 'Lassen',
 'San Luis Obispo',
 'Los Angeles',
 'San Francisco',
 'Marin',
 'San Diego',
 'Placer',
 'San Diego',
 'San Diego',
 'San Diego',
 'Los Angeles',
 'Santa Clara',
 'San Mateo',
 'San Diego',
 'Solano',


In [7]:
brewery_df_filter["County"] = brewcounties
brewery_df_filter["State"] = brewstates
brewery_df_filter["CountyID"] = brewcountycodes

brewery_df_filter

,BreweryName,BreweryType,Latitude,Longitude,County,State,CountyID
0,Avondale Brewing Co,micro,33.524521,-86.774322,Jefferson,AL,01073
1,Trim Tab Brewing,micro,33.512849,-86.791400,Jefferson,AL,01073
2,Yellowhammer Brewery,micro,34.727752,-86.593201,Madison,AL,01089
3,Bearpaw River Brewing Co,micro,61.575269,-149.412710,Matanuska-Susitna,AK,02170
4,King Street Brewing Co,micro,61.138489,-149.879076,Anchorage,AK,02020
...,...,...,...,...,...,...,...
4744,Public Craft Brewing Co,micro,42.583154,-87.819751,Kenosha,WI,55059
4745,Stevens Point Brewery,regional,44.510263,-89.573981,Portage,WI,55097
4746,Titletown Brewing Co,brewpub,44.518920,-88.018517,Brown,WI,55009
4747,Black Tooth Brewing Co,micro,44.800895,-106.953506,Sheridan,WY,56033


In [8]:
brewery_df_filter.to_csv("breweries_v1.csv")

In [9]:
y = 40.59335825
x = -105.0678422

thing = cg.coordinates(x=x, y=y)

thing['Counties'][0]['STATE'] + thing['Counties'][0]['COUNTY']

'08069'

In [10]:
t1 = datetime.now()

In [11]:
t2 = datetime.now()

t2-t1

datetime.timedelta(0, 0, 12033)